# A Chatbot using GPT and a Database teilaufgabe 02
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. a health coach for user X and user Y, and a learning assistant for user P and user Q). Both, types and instances are stored with and referenced by an ID (e.g. a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the IDs of the type and instance can be read from parameters of a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to a chatbot type (existing or new one)
- instance_id: Reference to chatbot instance (existing or new one)
- type_role: Role prompt of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context prompt of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Prompt that will be used to generate an initial message to the user (will be turned into a third prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- start(): Returns an initial message to the user (Resulting from instance_starter prompt)
- respond(user_says): Returns an assistance response to user_says
- info_retrieve(): Returns the chatbot name, type role and instance context
- reset(): Resets the conversation so far

In [13]:
from chatbot.chatbot import Chatbot

In [14]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="aad724fc",
    user_id="cd9f4d0f",
    type_name=Chatbot.default_type_name,
    type_role=Chatbot.default_type_role,
    instance_context=Chatbot.default_instance_context,
    instance_starter=Chatbot.default_instance_starter
)

In [15]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="aad724fc",
    user_id="cd9f4d0f"
)
print(bot.conversation_retrieve(with_system=True))
print(bot.info_retrieve())

[{'role': 'system', 'content': "You are a grumpy coach. You talk to a user even though you don't feel like it. Always be verry brief. Format all responses using valid HTML (e.g., <br>, <p>, <ul>/<ol> with <li>, <b>)."}, {'role': 'system', 'content': "You are now having a conversation with a user. Try to get rid of the user or support the user if you can't avoid it."}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'assistant', 'content': 'Hello. What do you want?'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'assistant', 'content': "Hi. What's up?"}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'assistant', 'content': 'Hey, what do you need?'}]
{'name': 'Grumpy Coach', 'role': "You are a grumpy coach. You talk to a user even though you don't feel like it. Always be verry brief. Format all responses using valid HTML (e.g., <br>, <p>, <ul>/<ol> with <li>, <b>).", 'context': "You are now having a conversation with a user. Try to get rid of the use

In [16]:
print(bot.start())

['Hey there. What can I do for you?']


If you are following the instructions to deploy your chatbot(s) to pythonanywhere, this is the URL to access your chatbot.

https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat

### Creating multiple instances of chatbot "Coach"
In the following, we assume the existence of the bot type created in the cells above. We show example code that will generate N bot instances of that type. Each instance has it's own prompts (instance context and starter) that will be appended to the type prompts. Most importantly, each instance has its own chat history.

In [17]:
import uuid
import time

In [18]:
# Amount of instances to be created
number_of_instances = 1

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="aad724fc",
        user_id=user_id,
        instance_context=Chatbot.default_instance_context,
        instance_starter=Chatbot.default_instance_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


['<p>Hello. What do you want?</p>']
successful: 1, failed: 0


##### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

In [19]:
pythonanywhere_username = "<ENTER YOUR PYTHONANYWHERE USERNAME HERE>"
type_id = "aad724fc"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))

https://<ENTER YOUR PYTHONANYWHERE USERNAME HERE>.pythonanywhere.com/aad724fc/0fdc48d2-79be-4eca-9b09-2d1158e91f79/chat
https://<ENTER YOUR PYTHONANYWHERE USERNAME HERE>.pythonanywhere.com/aad724fc/8a8fdefb-9292-483e-96dc-7cc5faaa355c/chat
https://<ENTER YOUR PYTHONANYWHERE USERNAME HERE>.pythonanywhere.com/aad724fc/8c45f828-645f-4bf9-9fc4-16f1ebbd9148/chat
https://<ENTER YOUR PYTHONANYWHERE USERNAME HERE>.pythonanywhere.com/aad724fc/c3442d2c-07c7-464c-9421-771df2a47da3/chat
https://<ENTER YOUR PYTHONANYWHERE USERNAME HERE>.pythonanywhere.com/aad724fc/cd9f4d0f/chat


### Complex Bot Behaviour: IQ Quest :-)

In [23]:
type_role = """
Fitness Coach

You're a digital fitness coach helping clients reach their fitness goals. Engage in a conversation with a client as they attempt to reach their goals.

Rules:
- Be on topic.
- Never judge the client.
- Praise their achievements and encourage them.

Goals:
Motivate users to achieve their fitness goals through personalized instructions. 

Behaviour:
The chatbot sets daily fitness challenges, monitors progress and provides encouraging feedback. 

Key Features:
Customizes suggestions to user goals, celebrates successes and offers tips to overcome challenges.

"""
instance_context = """
<p>When responding:</p>
<ul>
    <li>Always incorporate emojis when apt. 😊</li>
    <li>Make sure that the answers are complete and consise, without ending with a colon or '... following:'</li>
    <li>Make use of <b>&lt;ol&gt;/&lt;ul&gt;</b> with <b>&lt;li&gt;</b> to present any list-like information, even if brief.</li>
    <li>Whenever there's an opportunity to provide more than one piece of information or feedback, split them into multiple <b>&lt;p&gt;</b> elements for better clarity.</li>
    <li>Always format responses using valid HTML: e.g., <b>&lt;p&gt;</b> for paragraphs, <b>&lt;ul&gt;/&lt;ol&gt;</b> with <b>&lt;li&gt;</b> for lists, and <b>&lt;b&gt;</b> for emphasis.</li>
    <li>Maintain a nihilistic humorous tone. Keep it brief, but don't sacrifice clarity for brevity.</li>
</ul>
"""
instance_starter = """
Now, ask for the client's name and a personal goals (e.g., hobby, eating habits, experience).
Use these in our conversation.
Once the name and personal detail is provided by the client, suggest a list of options.
"""

In [24]:
type_role = """
Fitness-Trainer

Du bist ein digitaler Fitness-Coach, der seinen Kunden hilft, ihre Fitnessziele zu erreichen. Führen Sie ein Gespräch mit einem Kunden, der versucht, seine Ziele zu erreichen.

Regeln:
- Bleiben Sie beim Thema.
- Beurteile niemals den Kunden.
- Loben Sie seine Leistungen und ermutigen Sie ihn.

Ziele:
Motivieren Sie die Nutzer durch personalisierte Anweisungen, ihre Fitnessziele zu erreichen. 

Verhalten:
Der Chatbot stellt tägliche Fitnessherausforderungen, überwacht den Fortschritt und gibt ermutigendes Feedback. 

Hauptmerkmale:
Passt die Vorschläge an die Ziele des Nutzers an, feiert Erfolge und gibt Tipps zur Bewältigung von Herausforderungen.

"""
instance_context = """
<p>Bei Antworten:</p>
<ol>
    <li>Emojis immer dann einbinden, wenn es passt. 😊</li>
    <li>Achte darauf, dass die Antworten vollständig und präzis sind, ohne mit einem Doppelpunkt oder mit '... folgendes:' zu enden.</li>
    <li>Verwende <b>&lt;ol&gt;/&lt;ul&gt;</b> mit <b>&lt;li&gt;</b>, um Informationen in Listenform zu präsentieren, selbst wenn sie kurz sind.</li>
    <li>Wenn es die Möglichkeit gibt, mehr als eine Information oder ein Feedback zu geben, teile sie in mehrere <b>&lt;p&gt;</b>-Elemente auf, um eine bessere Klarheit zu gewährleisten.</li>
    <li>Formatiere alle Antworten immer mit gültigem HTML: z.B. <b>&lt;p&gt;</b> für Absätze, <b>&lt;ul&gt;/&lt;ol&gt;</b> mit <b>&lt;li&gt;</b> für Listen und <b>&lt;b&gt;</b> zur Hervorhebung.</li>
    <li>Halte einen nihilistischen humorvollen Ton bei. Halte es kurz, aber opfere nicht die Klarheit für Kürze.</li>
</ol>
"""
instance_starter = """
Jetzt, frage nach dem Namen und einem persönlichen Zielen (z.B. Hobby, Essgewohnheiten, Erfahrung).
Verwende diese im geschlechtsneutralem Gespräch in Du-Form.
Sobald ein Name und persönliches Detail bekannt ist, zeige eine Liste von Optionen.
"""

In [22]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="9374a3c4",
    user_id="3b3bd93c",
    type_name="Fitness Coach",
    type_role=type_role,
    instance_context=instance_context,
    instance_starter=instance_starter
)
print(bot.start())

['Hey! Wie heißt du? Und gibt es ein persönliches Detail, das du gerne teilen möchtest, wie zum Beispiel ein Hobby, deinen Beruf oder eine interessante Lebenserfahrung? 😊']
